In [ ]:
# Ячейка 1: Импорт библиотек
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
# Ячейка 2: Определение функции
def classification_training(data):
    """
    Функция для обучения модели случайного леса с подбором гиперпараметров
    """

    # 1) Трансформируем целевую переменную в бинарную
    data = data.copy()
    data['mental_wellness_binary'] = (data['mental_wellness_index_0_100'] >= 15).astype(int)

    # Подготовка признаков
    exclude_columns = ['user_id', 'mental_wellness_index_0_100', 'mental_wellness_binary']
    feature_columns = [col for col in data.columns if col not in exclude_columns]

    X = data[feature_columns]
    y = data['mental_wellness_binary']

    # Разделение данных
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # 2) Сетка гиперпараметров
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 10, 15, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    }

    # 3) Создаем модель
    rf = RandomForestClassifier(random_state=42)

    # 4) Поиск оптимальных параметров
    grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # 5) Обучение финальной модели
    best_rf = grid_search.best_estimator_

    # 6) Вычисление метрик
    y_pred = best_rf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Вывод результата в требуемом формате
    print(f"RF: {accuracy:.4f}; {f1:.4f}")

In [ ]:
# Ячейка 3: Загрузка данных и выполнение
df = pd.read_csv('cleaned_DB_3.csv')
classification_training(df)